# Amazon Fine Food Reviews Analysis


Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews <br>

EDA: https://nycdatascience.com/blog/student-works/amazon-fine-foods-visualization/


The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.<br>

Number of reviews: 568,454<br>
Number of users: 256,059<br>
Number of products: 74,258<br>
Timespan: Oct 1999 - Oct 2012<br>
Number of Attributes/Columns in data: 10 

Attribute Information:

1. Id
2. ProductId - unique identifier for the product
3. UserId - unqiue identifier for the user
4. ProfileName
5. HelpfulnessNumerator - number of users who found the review helpful
6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
7. Score - rating between 1 and 5
8. Time - timestamp for the review
9. Summary - brief summary of the review
10. Text - text of the review


#### Objective:
Given a review, determine whether the review is positive (rating of 4 or 5) or negative (rating of 1 or 2).

<br>
[Q] How to determine if a review is positive or negative?<br>
<br> 
[Ans] We could use Score/Rating. A rating of 4 or 5 can be cosnidered as a positive review. A rating of 1 or 2 can be considered as negative one. A review of rating 3 is considered nuetral and such reviews are ignored from our analysis. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.




# [1]. Reading Data

## [1.1] Loading the data

The dataset is available in two forms
1. .csv file
2. SQLite Database

In order to load the data, We have used the SQLITE dataset as it is easier to query the data and visualise the data efficiently.
<br> 

Here as we only want to get the global sentiment of the recommendations (positive or negative), we will purposefully ignore all Scores equal to 3. If the score is above 3, then the recommendation wil be set to "positive". Otherwise, it will be set to "negative".

In [80]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [42]:
# using SQLite Table to read data.
con = sqlite3.connect('database.sqlite') 

# filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
# SELECT * FROM Reviews WHERE Score != 3 LIMIT 500000, will give top 500000 data points
# you can change the number to any other number based on your computing power

# filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 500000""", con) 
# for tsne assignment you can take 5k data points

filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 10000""", con) 

# Give reviews with Score>3 a positive rating(1), and reviews with a score<3 a negative rating(0).
def partition(x):
    if x < 3:
        return 0
    return 1

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (10000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [43]:
display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", con)

In [44]:
print(display.shape)
display.head()

(80668, 7)


,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,#oc-R115TNMSPFT9I7,B005ZBZLT4,Breyton,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ESG,"Louis E. Emory ""hoppy""",1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B005ZBZLT4,Kim Cieszykowski,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ESG,Penguin Chick,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBEV0,Christopher P. Presta,1348617600,1,I didnt like this coffee. Instead of telling y...,2


In [45]:
display[display['UserId']=='AZY10LLTJ71NX']

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
80638,AZY10LLTJ71NX,B001ATMQK2,"undertheshrine ""undertheshrine""",1296691200,5,I bought this 6 pack because for the price tha...,5


In [46]:
display['COUNT(*)'].sum()

393063

#  [2] Exploratory Data Analysis

## [2.1] Data Cleaning: Deduplication

It is observed (as shown in the table below) that the reviews data had many duplicate entries. Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data.  Following is an example:

In [47]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


As it can be seen above that same user has multiple reviews with same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text and on doing analysis it was found that <br>
<br> 
ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies, 8.82-Ounce Packages (Pack of 8)<br>
<br> 
ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on<br>

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.<br>

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [48]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [49]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(9564, 10)

In [50]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

95.64

<b>Observation:-</b> It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows too are removed from calcualtions

In [51]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", con)

display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [52]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [53]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(9564, 10)


1    7976
0    1588
Name: Score, dtype: int64

#  [3] Preprocessing

## [3.1].  Preprocessing Review Text

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews

In [54]:
# printing some random reviews
sent_0 = final['Text'].values[0]
print(sent_0)
print("="*50)

sent_1000 = final['Text'].values[1000]
print(sent_1000)
print("="*50)

sent_1500 = final['Text'].values[1500]
print(sent_1500)
print("="*50)

sent_4900 = final['Text'].values[4900]
print(sent_4900)
print("="*50)

We have used the Victor fly bait for 3 seasons.  Can't beat it.  Great product!
15 month old loves to eat them on the go! They seem great for a healthy, quick, and easy snack!
These chips are truly amazing. They have it all. They're light, crisp, great tasting, nice texture, AND they're all natural... AND low in fat and sodium! Need I say more? I recently bought a bag of them at a regular grocery store, and couldn't belive my taste buds. That's why I excited why I saw them here on Amazon, and decided to buy a case!
These tablets definitely made things sweeter -- like lemons, limes, and grapefruit.  But it wasn't to the point of sheer amazement.  They also had an interesting effect on cheeses and vinegar, but still did virtually nothing for beer and wine. The tablets are a bit pricey but they do work.  If you've got extra money, sure, give them a try, but if you're looking for some amazing way to get your kids to eat broccoli or something along those lines then this is not the answer. F

In [55]:
# remove urls from text python: https://stackoverflow.com/a/40823105/4084039
sent_0 = re.sub(r"http\S+", "", sent_0)
sent_1000 = re.sub(r"http\S+", "", sent_1000)
sent_150 = re.sub(r"http\S+", "", sent_1500)
sent_4900 = re.sub(r"http\S+", "", sent_4900)

print(sent_0)

We have used the Victor fly bait for 3 seasons.  Can't beat it.  Great product!


In [56]:
# https://stackoverflow.com/questions/16206380/python-beautifulsoup-how-to-remove-all-tags-from-an-element
from bs4 import BeautifulSoup

soup = BeautifulSoup(sent_0, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1000, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1500, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_4900, 'lxml')
text = soup.get_text()
print(text)

We have used the Victor fly bait for 3 seasons.  Can't beat it.  Great product!
15 month old loves to eat them on the go! They seem great for a healthy, quick, and easy snack!
These chips are truly amazing. They have it all. They're light, crisp, great tasting, nice texture, AND they're all natural... AND low in fat and sodium! Need I say more? I recently bought a bag of them at a regular grocery store, and couldn't belive my taste buds. That's why I excited why I saw them here on Amazon, and decided to buy a case!
These tablets definitely made things sweeter -- like lemons, limes, and grapefruit.  But it wasn't to the point of sheer amazement.  They also had an interesting effect on cheeses and vinegar, but still did virtually nothing for beer and wine. The tablets are a bit pricey but they do work.  If you've got extra money, sure, give them a try, but if you're looking for some amazing way to get your kids to eat broccoli or something along those lines then this is not the answer. F

In [57]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [58]:
sent_1500 = decontracted(sent_1500)
print(sent_1500)
print("="*50)

These chips are truly amazing. They have it all. They are light, crisp, great tasting, nice texture, AND they are all natural... AND low in fat and sodium! Need I say more? I recently bought a bag of them at a regular grocery store, and could not belive my taste buds. That is why I excited why I saw them here on Amazon, and decided to buy a case!


In [59]:
#remove words with numbers python: https://stackoverflow.com/a/18082370/4084039
sent_0 = re.sub("\S*\d\S*", "", sent_0).strip()
print(sent_0)

We have used the Victor fly bait for  seasons.  Can't beat it.  Great product!


In [60]:
#remove spacial character: https://stackoverflow.com/a/5843547/4084039
sent_1500 = re.sub('[^A-Za-z0-9]+', ' ', sent_1500)
print(sent_1500)

These chips are truly amazing They have it all They are light crisp great tasting nice texture AND they are all natural AND low in fat and sodium Need I say more I recently bought a bag of them at a regular grocery store and could not belive my taste buds That is why I excited why I saw them here on Amazon and decided to buy a case 


In [61]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [62]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(final['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|████████████████████████████████████████████████████████████████████████████| 9564/9564 [00:03<00:00, 2717.41it/s]


In [63]:
len(preprocessed_reviews)

9564

# one hot encoding using bow of top 1000 features

In [64]:
count_vect = CountVectorizer(min_df=10,max_features=1000) #in scikit-learn CountVectorizer(min_df=10, max_features=500) 
X=count_vect.fit(preprocessed_reviews)

In [65]:
top_feat=count_vect.get_feature_names()

In [66]:
top_feat[1]

'absolutely'

In [67]:
word=[]
for sent in preprocessed_reviews:
    wrds = sent.split()
    for wrd in wrds:
        word.append(wrd)
    

In [68]:
feat=np.random.rand(1000)
for i in range(1000):
    count=0
    for wrd in word:
        if wrd==top_feat[i]:
            count=count+1
    feat[i]=count
        

In [69]:
sort_feat=np.argsort(feat)

In [70]:
sort_feat

array([678, 685,  84, 997, 246, 531, 382, 232, 860, 741, 828,   9, 912,
       428, 392, 768, 749, 694, 585, 787, 794, 861, 990, 154, 960,  19,
       738,  50, 239, 841, 354, 179, 725, 536, 249, 346, 868, 153, 661,
       440, 780, 873, 786, 580, 106, 878, 675, 957, 148, 459, 830, 439,
       867, 862, 508, 165, 849, 680, 557, 314, 197, 250, 946, 743, 839,
       337, 670, 825, 461,   3, 520, 758, 138, 700, 334, 151, 590, 949,
       945, 827, 240, 426, 668, 404,  12, 416, 345, 636,  69, 379,  71,
       176, 543, 798, 115, 582, 612, 968, 917, 777, 485, 433, 944, 526,
       851, 534, 649, 614, 271, 628, 189, 952, 318, 457,  67, 163,  99,
       381, 996, 144, 684, 927, 299, 929, 125, 605, 998, 976, 848, 507,
       815, 521, 625, 645, 129, 400, 795, 291, 871, 555, 104, 331, 778,
       311,  82,  65, 639, 658, 385,  41,  14,  44, 430, 112, 932, 116,
       516, 203, 522,  60, 644, 412, 986, 458, 746, 298, 689, 402, 316,
       538, 253, 647, 454, 514, 909, 858, 854, 135, 567, 727, 77

In [71]:
feat[574]

9474.0

In [72]:
sorted_list=[]
for i in sort_feat:
    sorted_list.append(top_feat[i])
    
    

In [73]:
sort_freq=dict( zip( sorted_list, sort_feat))

In [74]:
rank=[]
for sent in preprocessed_reviews:
    lst=[]
    wrds = sent.split()
    for wrd in wrds:
        if wrd in top_feat:
            lst.append(sort_freq[wrd])
    rank.append(lst)   

In [75]:
rank[0]

[935, 112, 574, 65, 383, 690]

In [76]:
len(rank)

9564

In [81]:
from sklearn.model_selection import train_test_split

p= final['Score']
X_train, X_test, y_train, y_test = train_test_split(rank, p, test_size=0.3, random_state=0)


In [82]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)


Using TensorFlow backend.


In [83]:
print(X_train[1])
print(type(X_train[1]))
print(len(X_train[1]))

[981, 539, 825, 593, 306, 936, 150, 920, 353, 592, 471, 435, 150, 797, 52, 150, 419, 539, 757, 61, 150, 196, 506, 286]
<class 'list'>
24


In [84]:
# truncate and/or pad input sequences
max_review_length = 600
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

print(X_train.shape)
print(X_train[1])

(6694, 600)
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   

In [86]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(1000+1, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 600, 32)           32032     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 85,333
Trainable params: 85,333
Non-trainable params: 0
_________________________________________________________________
None


In [87]:
model.fit(X_train, y_train, nb_epoch=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
6694/6694 [==============================] - ETA: 8:48 - loss: 0.6927 - acc: 0.640 - ETA: 4:49 - loss: 0.6909 - acc: 0.710 - ETA: 3:29 - loss: 0.6885 - acc: 0.760 - ETA: 2:49 - loss: 0.6864 - acc: 0.769 - ETA: 2:25 - loss: 0.6830 - acc: 0.793 - ETA: 2:08 - loss: 0.6797 - acc: 0.804 - ETA: 1:56 - loss: 0.6774 - acc: 0.799 - ETA: 1:47 - loss: 0.6722 - acc: 0.810 - ETA: 1:40 - loss: 0.6678 - acc: 0.812 - ETA: 1:34 - loss: 0.6620 - acc: 0.814 - ETA: 1:29 - loss: 0.6525 - acc: 0.822 - ETA: 1:25 - loss: 0.6423 - acc: 0.825 - ETA: 1:22 - loss: 0.6289 - acc: 0.826 - ETA: 1:18 - loss: 0.6283 - acc: 0.821 - ETA: 1:16 - loss: 0.6131 - acc: 0.824 - ETA: 1:13 - loss: 0.6080 - acc: 0.821 - ETA: 1:11 - loss: 0.6001 - acc: 0.820 - ETA: 1:09 - loss: 0.5857 - acc: 0.826 - ETA: 1:07 - loss: 0.5788 - acc: 0.826 - ETA: 1:06 - loss: 0.5734 - acc: 0.825 - ETA: 1:05 - loss: 0.5763 - acc: 0.819 - ETA: 1:03 - loss: 0.5704 - acc: 0.820 - ETA: 1:02 - loss

6694/6694 [==============================] - ETA: 56s - loss: 0.4649 - acc: 0.82 - ETA: 56s - loss: 0.3598 - acc: 0.84 - ETA: 56s - loss: 0.3241 - acc: 0.84 - ETA: 56s - loss: 0.3135 - acc: 0.86 - ETA: 55s - loss: 0.2990 - acc: 0.86 - ETA: 57s - loss: 0.3726 - acc: 0.86 - ETA: 57s - loss: 0.3639 - acc: 0.86 - ETA: 56s - loss: 0.3416 - acc: 0.87 - ETA: 55s - loss: 0.3342 - acc: 0.87 - ETA: 55s - loss: 0.3279 - acc: 0.87 - ETA: 54s - loss: 0.3197 - acc: 0.87 - ETA: 55s - loss: 0.3167 - acc: 0.87 - ETA: 54s - loss: 0.3073 - acc: 0.87 - ETA: 54s - loss: 0.3027 - acc: 0.87 - ETA: 53s - loss: 0.2967 - acc: 0.88 - ETA: 52s - loss: 0.3011 - acc: 0.87 - ETA: 52s - loss: 0.3002 - acc: 0.87 - ETA: 51s - loss: 0.3032 - acc: 0.87 - ETA: 51s - loss: 0.3050 - acc: 0.87 - ETA: 50s - loss: 0.3061 - acc: 0.87 - ETA: 49s - loss: 0.3008 - acc: 0.87 - ETA: 49s - loss: 0.2979 - acc: 0.87 - ETA: 48s - loss: 0.2991 - acc: 0.87 - ETA: 48s - loss: 0.2956 - acc: 0.87 - ETA: 48s - loss: 0.2942 - acc: 0.87 - ETA: 

6694/6694 [==============================] - ETA: 1:04 - loss: 0.2080 - acc: 0.906 - ETA: 1:00 - loss: 0.1787 - acc: 0.929 - ETA: 58s - loss: 0.2188 - acc: 0.901 - ETA: 1:02 - loss: 0.2373 - acc: 0.902 - ETA: 1:07 - loss: 0.2236 - acc: 0.909 - ETA: 1:07 - loss: 0.2340 - acc: 0.901 - ETA: 1:07 - loss: 0.2224 - acc: 0.910 - ETA: 1:06 - loss: 0.2098 - acc: 0.919 - ETA: 1:04 - loss: 0.1947 - acc: 0.927 - ETA: 1:02 - loss: 0.1796 - acc: 0.934 - ETA: 1:01 - loss: 0.1835 - acc: 0.933 - ETA: 59s - loss: 0.1790 - acc: 0.936 - ETA: 58s - loss: 0.1809 - acc: 0.93 - ETA: 58s - loss: 0.1808 - acc: 0.93 - ETA: 58s - loss: 0.1808 - acc: 0.93 - ETA: 57s - loss: 0.1828 - acc: 0.92 - ETA: 56s - loss: 0.1772 - acc: 0.93 - ETA: 55s - loss: 0.1767 - acc: 0.93 - ETA: 54s - loss: 0.1769 - acc: 0.93 - ETA: 53s - loss: 0.1838 - acc: 0.92 - ETA: 52s - loss: 0.1833 - acc: 0.93 - ETA: 51s - loss: 0.1806 - acc: 0.93 - ETA: 50s - loss: 0.1808 - acc: 0.93 - ETA: 49s - loss: 0.1801 - acc: 0.93 - ETA: 49s - loss: 0.17

6694/6694 [==============================] - ETA: 54s - loss: 0.1150 - acc: 0.95 - ETA: 54s - loss: 0.1614 - acc: 0.94 - ETA: 53s - loss: 0.1829 - acc: 0.93 - ETA: 53s - loss: 0.1910 - acc: 0.92 - ETA: 52s - loss: 0.1858 - acc: 0.92 - ETA: 52s - loss: 0.1933 - acc: 0.91 - ETA: 51s - loss: 0.1824 - acc: 0.92 - ETA: 51s - loss: 0.1905 - acc: 0.91 - ETA: 51s - loss: 0.1825 - acc: 0.92 - ETA: 50s - loss: 0.1843 - acc: 0.92 - ETA: 50s - loss: 0.1775 - acc: 0.92 - ETA: 49s - loss: 0.1766 - acc: 0.92 - ETA: 49s - loss: 0.1737 - acc: 0.92 - ETA: 48s - loss: 0.1727 - acc: 0.92 - ETA: 48s - loss: 0.1720 - acc: 0.92 - ETA: 47s - loss: 0.1699 - acc: 0.92 - ETA: 47s - loss: 0.1664 - acc: 0.92 - ETA: 46s - loss: 0.1656 - acc: 0.92 - ETA: 46s - loss: 0.1653 - acc: 0.92 - ETA: 45s - loss: 0.1679 - acc: 0.92 - ETA: 44s - loss: 0.1672 - acc: 0.93 - ETA: 44s - loss: 0.1642 - acc: 0.93 - ETA: 43s - loss: 0.1654 - acc: 0.93 - ETA: 43s - loss: 0.1593 - acc: 0.93 - ETA: 42s - loss: 0.1575 - acc: 0.93 - ETA: 

6694/6694 [==============================] - ETA: 55s - loss: 0.1418 - acc: 0.93 - ETA: 54s - loss: 0.1199 - acc: 0.96 - ETA: 54s - loss: 0.1097 - acc: 0.96 - ETA: 55s - loss: 0.1072 - acc: 0.96 - ETA: 54s - loss: 0.1225 - acc: 0.95 - ETA: 54s - loss: 0.1253 - acc: 0.95 - ETA: 53s - loss: 0.1270 - acc: 0.95 - ETA: 52s - loss: 0.1170 - acc: 0.95 - ETA: 52s - loss: 0.1095 - acc: 0.96 - ETA: 51s - loss: 0.1189 - acc: 0.95 - ETA: 51s - loss: 0.1146 - acc: 0.95 - ETA: 50s - loss: 0.1116 - acc: 0.95 - ETA: 50s - loss: 0.1107 - acc: 0.95 - ETA: 49s - loss: 0.1062 - acc: 0.95 - ETA: 49s - loss: 0.1135 - acc: 0.95 - ETA: 48s - loss: 0.1116 - acc: 0.95 - ETA: 48s - loss: 0.1192 - acc: 0.95 - ETA: 47s - loss: 0.1254 - acc: 0.95 - ETA: 46s - loss: 0.1237 - acc: 0.95 - ETA: 46s - loss: 0.1227 - acc: 0.95 - ETA: 45s - loss: 0.1250 - acc: 0.94 - ETA: 45s - loss: 0.1262 - acc: 0.94 - ETA: 44s - loss: 0.1258 - acc: 0.94 - ETA: 44s - loss: 0.1281 - acc: 0.94 - ETA: 43s - loss: 0.1310 - acc: 0.94 - ETA: 

6694/6694 [==============================] - ETA: 1:00 - loss: 0.1306 - acc: 0.953 - ETA: 59s - loss: 0.1194 - acc: 0.953 - ETA: 59s - loss: 0.1288 - acc: 0.95 - ETA: 59s - loss: 0.1111 - acc: 0.96 - ETA: 58s - loss: 0.1071 - acc: 0.96 - ETA: 57s - loss: 0.1132 - acc: 0.96 - ETA: 57s - loss: 0.1260 - acc: 0.95 - ETA: 57s - loss: 0.1399 - acc: 0.95 - ETA: 56s - loss: 0.1364 - acc: 0.95 - ETA: 56s - loss: 0.1263 - acc: 0.95 - ETA: 55s - loss: 0.1220 - acc: 0.96 - ETA: 54s - loss: 0.1260 - acc: 0.96 - ETA: 54s - loss: 0.1264 - acc: 0.96 - ETA: 53s - loss: 0.1264 - acc: 0.95 - ETA: 52s - loss: 0.1258 - acc: 0.95 - ETA: 52s - loss: 0.1204 - acc: 0.96 - ETA: 51s - loss: 0.1243 - acc: 0.96 - ETA: 51s - loss: 0.1262 - acc: 0.96 - ETA: 50s - loss: 0.1275 - acc: 0.95 - ETA: 49s - loss: 0.1264 - acc: 0.95 - ETA: 49s - loss: 0.1257 - acc: 0.95 - ETA: 48s - loss: 0.1266 - acc: 0.95 - ETA: 48s - loss: 0.1284 - acc: 0.95 - ETA: 47s - loss: 0.1303 - acc: 0.95 - ETA: 47s - loss: 0.1317 - acc: 0.95 - ET